### Config stuff

In [1]:

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

In [2]:
spark = cc.startLocalCluster("factSales")
spark.getActiveSession()

# Fact transformations
This notebooks creates the sales fact table from scratch based on the operational source table "sales"
When creating a fact table always follow the listed steps in order.


#### 1 READ NECESSARY SOURCE TABLE(S) AND PERFORM TRANSFORMATIONS
**When reading from the source table make sure you include all data necessary:**
- to calculate the measure values
- the source table keys that you have to use to lookup the correct surrogate keys in the dimension tables.

**If more than one table is needed to gather the necesary information you can opt for one of two strategies:**
- Use a select query when reading from the jdbc source with the spark.read operation. Avoid complex queries because the operational database needs a lot of resources to run those queries.
- Perform a spark.read operation for each table separately and join the tables within Spark. The joins will take place on the cluster instead of the database. You limit the database recources used, but there can be a significant overhead of unnecessary data tranferred to the cluster.


In this case we just rename Amount and create a default count_mv column.
The transformations are minimal. In reality, transformations can be far more complex. If so, it can be advisable to work out the transforms in more then one step.*



In [3]:
cc.set_connection("default")
sale_src_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable", "sales") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "Order_ID") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()\


#sale_src_df.show(20)


#### 2 MAKE DIMENSION TABLES AVAILABLE AS VIEWS

In [4]:
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")
dim_salesrep = spark.read.format("delta").load("spark-warehouse/dimsalesrep/")
dim_date.createOrReplaceTempView("dimDate")
dim_salesrep.createOrReplaceTempView("dimSalesRep")


#### 3 Build the fact table

Within the creation of a fact table always perform these two tasks:
1.   Include the measures of the fact
2. Use the dimension tables to look up the surrogate keys that correspond with the natural key value. In case of SCD2 dimension use the scd_start en scd_end to find the correct version of the data in the dimension


In [5]:
sale_src_df.createOrReplaceTempView("sales_source")
salesFactFromSource = spark.sql("select src.Order_ID as OrderID, dd.dateSk as dateSK, ds.salesrepSK, 1 as count_mv, src.amount as revenue_mv, md5(concat(src.Order_ID,dd.dateSk,ds.salesrepSK, 1, src.amount)) as md5 \
          from sales_source as src \
          left outer join dimdate as dd on src.Order_Date = cast(dd.CalendarDate as DATE) \
           left outer join dimSalesRep as ds \
                      on src.SalesRepID = ds.SalesRepID \
                      and src.Order_Date > ds.scd_start \
                      and src.Order_Date <= ds.scd_end")

salesFactFromSource.show(10)
salesFactFromSource.createOrReplaceTempView("factSales_new")

+-------+------+--------------------+--------+----------+--------------------+
|OrderID|dateSK|          salesrepSK|count_mv|revenue_mv|                 md5|
+-------+------+--------------------+--------+----------+--------------------+
|      1|   650|e29017f3-ee76-462...|       1| 851804379|6e8de602838d31192...|
|      2|  1369|e29017f3-ee76-462...|       1| 683057055|48ec86cac5a62d038...|
|      3|   920|e29017f3-ee76-462...|       1|1732115679|de3629828fb3a84b6...|
|      4|   604|e29017f3-ee76-462...|       1|1275042249|1eab0e382551878fa...|
|      5|   897|e29017f3-ee76-462...|       1| 694153767|2dd11e3404f663f9a...|
|      6|   812|e29017f3-ee76-462...|       1|1959464599|d25ac79b971382a47...|
|      7|   421|e29017f3-ee76-462...|       1|1170677605|61d43c8ee34339b4b...|
|      8|   691|e29017f3-ee76-462...|       1|1588502393|329cb83447f61cdcb...|
|      9|  1254|e29017f3-ee76-462...|       1|1173163372|76f9615aff1703787...|
|     10|  1311|e29017f3-ee76-462...|       1| 78868

## Initial load
The first time loading the fact table perform a FULL load. All data is written to the Delta Table.
After initial load the code line has to be disabled

In [7]:

salesFactFromSource.write.format("delta").mode("overwrite").saveAsTable("factSales")



## Incremental load
When previous runs where performend you can opt for a 'faster' incremental run that only writes away changes. UPDATES and INSERTS are performed in one run.
In our solution we use an md5 based on all fields in the source table to detect changes. This is not the most efficient way to detect changes. A better way is to use a timestamp field in the source table and use that to detect changes. This is not implemented in this example.

In [34]:
dt_factSales = DeltaTable.forPath(spark,".\spark-warehouse\\factsales")
dt_factSales.toDF().createOrReplaceTempView("factSales_current")
#Merge to perform updates (TODO: Implement md5 strategy)

result = spark.sql("MERGE INTO factSales_current AS target \
      using factSales_new AS source ON target.orderID = source.orderID \
      WHEN MATCHED and source.MD5<>target.MD5 THEN UPDATE SET * \
      WHEN NOT MATCHED THEN INSERT *")

result.show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                1|               0|               0|                1|
+-----------------+----------------+----------------+-----------------+


In [6]:
# IMPORTANT: ALWAYS TEST THE CREATED CODE.
# In this example I changed order 498 in the operational database and checked the change after the run.
# spark.sql("select * from factsales f join dimsalesrep ds on f.salesrepSK = ds.salesrepSK where OrderID = 192  ").show()
spark.sql("select count(*) from factsales").show()
spark.sql("select * from factsales where orderId=1").show()



AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `factsales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 21;
'Aggregate [unresolvedalias(count(1), None)]
+- 'UnresolvedRelation [factsales], [], false


### Checking the history of your delta fact table

In [ ]:
# The history information is derived from the delta table log files. They contain a lot of information of all the actions performed on the table. In this case it tells us something about de merge operations. You can find statistics about the update and insert counts in the document.

fact.history().show(10,False)

In [ ]:
spark.stop()